In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import torch.nn.init as init
from torch import maximum
from torch import minimum
import Hawkes as hk
import matplotlib.pyplot as plt
import random
from autograd import grad

In [22]:
# Paramètres :
mu, alpha, beta = 0.05, 1/4, 4
T = 60000

model = hk.simulator()
model.set_kernel('exp')
model.set_baseline('const')
theta = {'mu':mu, 'alpha':alpha, 'beta':beta}
model.set_parameter(theta)
H_T = model.simulate([0,T])
print(len(H_T))
print(type(H_T))

4047
<class 'numpy.ndarray'>


In [23]:
class ExcitationKernel(nn.Module):
    def _init_(self, p=100):
        super()._init_()
        self.A1 = nn.Linear(1, p)
        self.A2 = nn.Linear(p, 1)

        # Initialisation des poids des couches linéaires
        init.uniform_(self.A1.weight, a=0, b=0.5)
        init.uniform_(self.A2.weight, a=-0.5, b=0)

    def forward(self, x):
        print(x)
        x = F.relu(self.A1(x))
        print(x)
        x = self.A2(x)
        print(x)
        return torch.exp(x)

def phi(t, b1, W2, W1, b2):
    #print('ici0', np.exp(b2 + np.dot(W2, np.maximum(W1*t + b1, 0))))
    return torch.exp(b2 + np.dot(W2, maximum(W1*t + b1, 0)))

def calculer_lambda(t, mu, b1, W2, W1, b2, H_T):
    lambda_val = mu
    for tau in H_T:
        if tau < t:
            lambda_val += phi(t - tau, b1, W2, W1, b2)
        else:
            break
    return lambda_val

In [24]:

class LogLikelihood():
    def __init__(self, W1, W2, b1, b2, mu, phi,T):
        super(LogLikelihood, self).__init__()
        self.W1 = W1
        self.W2 = W2
        self.b1 = b1
        self.b2 = b2
        self.mu = mu
        self.phi = phi
        self.T=T

    def calculer_lambda(self, t, H_T):
        lambda_val = self.mu
        phi=self.phi
        for tau in H_T:
            if tau < t:
                lambda_val += phi(t - tau)
            else:
                break
        return torch.tensor(lambda_val)

    def calculate_integral(self, t):

        W1 = self.W1
        W2 = self.W2
        b1 = self.b1
        b2 = self.b2
        mu = self.mu
        phi = self.phi

        #W1 = np.array(W1)  
        #b1 = np.array(b1)  
        #W2 = np.array(W2)
    
        s_inflection_points = -b1 / W1.t()
        s_inflection_points = torch.sort(s_inflection_points).values
    
        integral_value = 0.0
        eps = 10**(-6)

# Find the largest subsequence of sorted inflection points within [0, t]

        s_l = torch.max(s_inflection_points[s_inflection_points <= t])
        s_u = torch.min(s_inflection_points[s_inflection_points >= 0.0])

        # Calculate the integral by summing over segments between consecutive inflection points
        for i in range(len(s_inflection_points) - 1):
            s_m = s_inflection_points[i]
            s_n = s_inflection_points[i + 1]
            if s_m > 0 and s_n < t:
                #print([W2[j] * W1[j] for j in range(len(W2)) if W1[j] * (s_n - eps) + b1[j] > 0])
                integral_value += (phi(s_n) - phi(s_m)) / (
                    torch.sum(torch.tensor(W2 * W1 *(W1 * (s_n - eps) + b1 > 0).float()))
                )

        integral_value += (phi(s_u) - phi(0)) / (
                    torch.sum(W2 * W1 *(W1 * (s_u - eps) + b1 > 0).float())
                )

        integral_value += (phi(t) - phi(s_l)) / (
                    torch.sum(W2 * W1 *(W1 * (t - eps) + b1 > 0).float())
                )
        #W1 = torch.tensor(W1, dtype=torch.float32)  
        #b1 = torch.tensor(b1, dtype=torch.float32)  
        #W2 = torch.tensor(W2, dtype=torch.float32)
        

        return integral_value
    
    def loss_final(self, t):
        loss_value = torch.tensor(0.0)
        mu = self.mu
        H_T=self.T
        if H_T.any():
            H = torch.tensor(np.array([t_n for t_n in H_T if t_n < t]))
            for i in range(1, H.size(0)):
                loss_value += torch.log(self.calculer_lambda(H[i], H)) - mu * (H[i] - H[i - 1]) - self.calculate_integral(t - H[i])
            loss_value += torch.log(self.calculer_lambda(H[0], H)) - mu * (H[0]) - self.calculate_integral(t - H[0])
        return loss_value

In [31]:


# Définir le réseau de neurones
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def init_weights(self):
        # Initialisation des poids de la couche d'entrée de manière aléatoire
        init.uniform(self.fc1.weight, 0.0,0.5)
        init.constant_(self.fc1.bias, 0.1)  # Initialisation des biais à zéro

        # Initialisation des poids de la couche de sortie de manière aléatoire
        init.uniform(self.fc2.weight, -0.5,0)
        init.constant_(self.fc2.bias, 0.1)  # Initialisation des biais à zéro

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = torch.exp(out)
        return out

# Définir les paramètres du réseau de neurones
input_size = 1
hidden_size = 100
output_size = 1

# Créer une instance du réseau de neurones
model = SimpleNN(input_size, hidden_size, output_size)
model.fc1.weight.requires_grad_(True)
model.fc1.bias.requires_grad_(True)


# Spécification des taux d'apprentissage pour chaque couche
learning_rate_fc1 = 1e-5  # Taux d'apprentissage pour la couche d'entrée
learning_rate_fc2 = 1e-2  # Taux d'apprentissage pour la couche de sortie

# Définition des optimiseurs pour chaque couche avec des taux d'apprentissage différents
optimizer_fc1 = optim.Adam(model.fc1.parameters(), lr=learning_rate_fc1)
optimizer_fc2 = optim.Adam(model.fc2.parameters(), lr=learning_rate_fc2)
mu=0.05



x=torch.tensor(H_T,dtype=torch.float32)
# # Boucle d'entraînement
for epoch in range(30):
        #optimizer_fc1.zero_grad()  # Remise à zéro des gradients
        #optimizer_fc2.zero_grad()
        phi_chap = lambda a: model(torch.tensor(np.array([a]), dtype=torch.float32)).item()

        # Passage avant : calcul de la perte
        loss = -LogLikelihood(model.state_dict()['fc1.weight'],model.state_dict()['fc2.weight'],model.state_dict()['fc1.bias'],model.state_dict()['fc1.bias'],mu,phi_chap,x).loss_final(60000)
    
        # Rétropropagation : calcul des gradients
        loss.requires_grad_(True)
        loss.backward()
        
        # Mise à jour des poids
        optimizer_fc1.step()
        optimizer_fc2.step()
        
        # Afficher la perte à chaque époque
        print(f'Epoch [{epoch+1}/{30}], Loss: {loss.item():.4f}')

Epoch [1/30], Loss: 4929.4902
Epoch [2/30], Loss: 4929.4902


KeyboardInterrupt: 